# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# Libraries
import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [2]:
# Import dataframe
path = 'data/Admission_Predict.csv'
admissions = pd.read_csv(path)

Remove trailing spaces at the end of the column names if there are any.


In [3]:
admissions.columns = admissions.columns.str.strip()

Let's evaluate the dataset by looking at the `head` function.

In [4]:
admissions.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [5]:
admissions.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [ ]:
admissions.set_index('Serial No.', drop = False)

Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [6]:
admissions[['GRE Score', 'CGPA']].duplicated().sum()

0

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [ ]:
admissions.set_index(['GRE Score', 'CGPA'], inplace = True)

# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [7]:
admissions.loc[(admissions['CGPA'] > 9) & (admissions['Research'] > 0)].shape[0]

101

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [8]:
admissions.loc[(admissions['CGPA'] > 9) & (admissions['SOP'] < 3.5)]['Chance of Admit'].mean()

0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [9]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    return (col - col.mean()) / col.std()

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [10]:
admissions['CGPA_std'] = standardize(admissions['CGPA'])
admissions['GRE_std'] = standardize(admissions['GRE Score'])
admissions['LOR_std'] = standardize(admissions['LOR'])

admissions.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.747900,1.753381,1.191647
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.991211,-0.063367,0.076740
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121034,0.455704,-1.038166
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.642597,-0.236391,-0.480713
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.233279,1.147798,-0.480713


We will generate the decision choice at random using the code below. Please run the cell.

In [11]:
# Libraries
from random import choices

In [18]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])
decision_choice

['CGPA_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std'

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [17]:
admissions['deciding_column'] = admissions.lookup(admissions.index, decision_choice)
admissions

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.747900,1.753381,1.191647,1.753381
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.991211,-0.063367,0.076740,0.076740
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121034,0.455704,-1.038166,0.455704
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.642597,-0.236391,-0.480713,-0.236391
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.233279,1.147798,-0.480713,-0.480713
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.735259,0.628727,0.076740,0.628727
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.851463,0.715239,0.076740,0.715239
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.415886,1.147798,1.191647,1.415886
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.303641,-0.409414,0.634193,0.634193


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [28]:
admissions.loc[admissions['deciding_column'] > 0.8, 'decision'] = 1 
admissions.loc[admissions['deciding_column'] <= 0.8, 'decision'] = 0 
admissions

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.747900,1.753381,1.191647,1.753381,1.0
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.991211,-0.063367,0.076740,0.076740,0.0
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121034,0.455704,-1.038166,0.455704,0.0
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.642597,-0.236391,-0.480713,-0.236391,0.0
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.233279,1.147798,-0.480713,-0.480713,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.735259,0.628727,0.076740,0.628727,0.0
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.851463,0.715239,0.076740,0.715239,0.0
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.415886,1.147798,1.191647,1.415886,1.0
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.303641,-0.409414,0.634193,0.634193,0.0


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [30]:
admissions.loc[admissions['decision'] == 1].shape[0]

91

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [36]:
import re
admissions.columns = admissions.columns.str.replace(r'[^a-zA-Z ]\s?',r'',regex=True).str.replace(' ', '_').str.lower()
admissions.head()

,serial_no,gre_score,toefl_score,university_rating,sop,lor,cgpa,research,chance_of_admit,cgpastd,grestd,lorstd,decidingcolumn,decision
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.747900,1.753381,1.191647,1.753381,1.0
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.991211,-0.063367,0.076740,0.076740,0.0
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121034,0.455704,-1.038166,0.455704,0.0
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.642597,-0.236391,-0.480713,-0.236391,0.0
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.233279,1.147798,-0.480713,-0.480713,0.0


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [44]:
admissions['gre_score'].loc[(admissions['university_rating'] >= 4), 'adjusted_gre'] = [i + 10 for i in admissions['gre_score']]



IndexingError: (0       True
1      False
2      False
3      False
4       True
       ...  
380    False
381    False
382     True
383    False
384     True
Name: university_rating, Length: 385, dtype: bool, 'adjusted_gre')

In [ ]:
#https://datatofish.com/if-condition-in-pandas-dataframe/